In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [20]:
from src.inference import get_feature_store

In [21]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from ),
    end_time=(fetch_data_to),
)

ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]


2025-03-05 15:02:44,583 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 15:02:44,599 INFO: Initializing external client
2025-03-05 15:02:44,599 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 15:02:45,736 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214681
Fetching data from 2025-02-04 20:02:44.582029+00:00 to 2025-03-05 19:02:44.582029+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (70.11s) 


In [22]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 21:00:00+00:00,2,0
1,2025-02-04 22:00:00+00:00,2,0
2,2025-02-04 23:00:00+00:00,2,0
3,2025-02-05 00:00:00+00:00,2,0
4,2025-02-05 01:00:00+00:00,2,0
...,...,...,...
174379,2025-03-05 12:00:00+00:00,263,100
174380,2025-03-05 13:00:00+00:00,263,108
174381,2025-03-05 14:00:00+00:00,263,136
174382,2025-03-05 15:00:00+00:00,263,153


In [23]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174384 entries, 0 to 174383
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174384 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174384 non-null  int32                  
 2   rides               174384 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 2.7 MB


In [24]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [25]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174384 entries, 0 to 174383
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174384 non-null  datetime64[us]
 1   pickup_location_id  174384 non-null  int32         
 2   rides               174384 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 2.7 MB


In [26]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [27]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,5,2,0,3,2,7,3,0,...,1,1,1,4,1,1,1,2,7,2025-02-27 21:00:00
1,1,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,106,2025-02-17 06:00:00
2,0,2,4,1,0,1,1,0,0,0,...,0,0,0,1,0,0,1,2,49,2025-03-03 15:00:00
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-23 09:00:00
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,194,2025-02-20 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,188,176,201,151,272,136,122,356,18,7,...,410,147,59,22,43,139,318,22,230,2025-02-15 20:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,120,2025-02-28 08:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,206,2025-02-25 22:00:00
250,1,1,5,1,0,0,0,2,2,0,...,4,3,0,2,1,2,0,4,243,2025-02-12 05:00:00


In [28]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 15:04:07,779 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 15:04:07,786 INFO: Initializing external client
2025-03-05 15:04:07,786 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 15:04:08,653 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214681
Fetching data from 2025-02-04 20:04:07.779757+00:00 to 2025-03-05 19:04:07.779757+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (57.41s) 


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
current_date

Timestamp('2025-03-05 20:04:07.779757+0000', tz='Etc/UTC')

In [30]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 21:00:00
1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,1,0,3,2025-03-04 21:00:00
2,5,1,0,2,0,3,0,0,0,0,...,2,3,6,2,1,5,3,3,4,2025-03-04 21:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 21:00:00
4,1,1,2,1,1,0,0,0,1,2,...,1,4,5,0,3,1,2,0,7,2025-03-04 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,259,2025-03-04 21:00:00
248,0,0,0,1,0,0,0,0,0,0,...,1,0,1,0,2,1,1,0,260,2025-03-04 21:00:00
249,38,18,9,6,4,0,1,0,1,7,...,27,36,26,28,58,46,34,25,261,2025-03-04 21:00:00
250,21,18,6,2,1,0,5,6,13,66,...,89,104,140,111,101,87,48,30,262,2025-03-04 21:00:00


In [31]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 15:05:17,944 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 15:05:17,952 INFO: Initializing external client
2025-03-05 15:05:17,952 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 15:05:18,550 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214681


In [32]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [33]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,7.0
3,6,0.0
4,7,1.0
...,...,...
247,259,0.0
248,260,0.0
249,261,28.0
250,262,78.0


In [34]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 237, 132, 161, 163,  79, 162, 142, 239, 230])